In [2]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display_html
import numpy as np

# Football data from wikipedia

The objective of this notebook is to extract information on British football teams from Wikipedia. This primarily utilises the wikipedia API to extract the data, then Beautiful soup is used to manipulate the data.

## Wikipedia API

The documentation for the wikipedia API is here:
https://www.mediawiki.org/wiki/API:Main_page

The inputs to the get request are:
- URL: the wikipedia API endpoint used throughout.
- TITLE: the title of the page to be scraped
- PARAMS: the API takes its input through parameters provided by the query string

In [3]:
S = requests.Session() # This object allows you to persist certain parameters across requests. Results in better performance.

URL = "https://en.wikipedia.org/w/api.php" # English wikipedia API endpoint

TITLE = "List of football clubs in England"

PARAMS = {
    'action': 'parse', # the action is to parse the content of the page
    'page': TITLE,
    'format': 'json'   # the API can return other formats but they are standardising to JSON
    #'UTF8':            # convert to UTF-8 encoding - does this work? Is it already UTF-8?
}

### Requesting data from a page
Next, we must request the data from the wiki page and turn it into a python object that we can easily manipulate. We must interpret the request data as JSON in order to convert it into __class 'dict'__.

The scraped data are also stored in a JSON file for future use so that we do not need to make the same request multiple times.

In [3]:
'''
# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS)
DATA = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

# Write to JSON file
with open('list_clubs_in_england.json', 'w') as f:
    json.dump(DATA, f)
'''

"\n# Get the response data as a python object. \nR = S.get(url=URL, params=PARAMS)\nDATA = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>\n\n# Write to JSON file\nwith open('list_clubs_in_england.json', 'w') as f:\n    json.dump(DATA, f)\n"

In [4]:
# Read JSON file
with open('list_clubs_in_england.json') as data_file:
    DATA = json.load(data_file)

### The data structure within the dictionary object:

In [5]:
DATA.keys()

dict_keys(['parse'])

In [6]:
parse=DATA['parse']
parse.keys()

dict_keys(['iwlinks', 'langlinks', 'sections', 'text', 'templates', 'categories', 'pageid', 'revid', 'displaytitle', 'links', 'externallinks', 'parsewarnings', 'properties', 'title', 'images'])

In [7]:
text = parse['text']
type(text['*'])

str

### Manipulating the html

#### Defining the function(s)
Here is the original code where the following functions are defined: https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/

We don't need to use the 'parse_url' function since we have to parse the URL differently. In future we should write our own version of the function though - essentially one function call to produce the desired table.

Currently this function does not quite work as required, the number of columns does not work automatically and must be manually entered.

In [37]:
#def parse_url(self, url):
#    response = requests.get(url)
#    soup = BeautifulSoup(response.text, 'lxml')
#    return [(table['id'],self.parse_html_table(table))\
#            for table in soup.find_all('table')]  

def parse_html_table(self, table, def_cols = -1):
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                if (def_cols == -1):
                    n_columns = len(td_tags)
                else:
                    n_columns = def_cols

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text().strip())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text().strip()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

#### Producing the pandas dataframe

In [15]:
# Parsing the HTML using Beautiful Soup
soup = BeautifulSoup(DATA['parse']['text']['*'], "html.parser")
# print(soup.prettify())

# Extract all of the tables from the soup and put them into separate elements of a list
alltables = soup.findAll('table')
print("Number of tables found : " , len(alltables))

Number of tables found :  31


In [78]:
# Call our function on each table within the list to produce a list of pandas tables
# We are only interested in the tables containing team names (tables 1-25)
alltab_parse=[parse_html_table(alltables[tab], alltables[tab], 5) for tab in range(1,25)] 

# Concatenate the results into a single pandas dataframe
result = pd.concat(alltab_parse).reset_index(drop=True)
result.head()

,Club,League/Division,Lvl,Nickname,Change from 2017–18
0,A.F.C. Aldermaston,Hellenic League Division One East,10.0,Atom Men,NaN
1,A.F.C. Blackpool,North West Counties League Division One North,10.0,Mechanics,NaN
2,A.F.C. Bournemouth,Premier League,1.0,Cherries,NaN
3,A.F.C. Bridgnorth,West Midlands (Regional) League Premier Division,10.0,Meadow Men,NaN
4,A.F.C. Croydon Athletic,Southern Counties East League Premier Division,9.0,Rams,NaN


In [17]:
result.keys()


Index(['Club', 'League/Division', 'Lvl', 'Nickname', 'Change from 2017–18'], dtype='object')

In [18]:
# There are 20 teams in the Premier League
prem_teams = result.loc[result['League/Division'] == 'Premier League']
len(prem_teams.index)

20

## Extract team information on Premier League

In [24]:
'''
# Extract data on all Premier League teams, convert from JSON to dictionary structure. Store each dictionary structure in a list
club_data_json = [(S.get(url=URL, params={'action': "parse",'page': row.Club,'format': "json"})).json() for row in prem_teams.itertuples()]

# Write to a file. Each teams data is written on a single line
with open('premier_league_club_data.json', 'w') as f:
    json.dump(club_data_json, f)
'''

In [49]:
with open('premier_league_club_data.json') as f:
    club_data_json = json.load(f)

In [59]:
# Place the first team page data into bs4
soup_club1 = BeautifulSoup(club_data_json[0]['parse']['text']['*'], "html.parser")

alltables_club1 = soup_club1.findAll("table")

html_content = str(alltables_club1[2])
display_html(html_content, raw=True)

No.,,Position,Player
1,,GK,Artur Boruc
2,,DF,Simon Francis (captain)[31]
3,,DF,Steve Cook (2nd vice-captain)
4,,MF,Dan Gosling
5,,DF,Nathan Aké
6,,MF,Andrew Surman (vice-captain)[31]
7,,MF,Marc Pugh
8,,MF,Jefferson Lerma
9,,FW,Lys Mousset
10,,MF,Jordon Ibe


In [60]:
dfs = pd.read_html(html_content)
dfs[1]

,0,1,2,3
0,No.,NaN,Position,Player
1,1,NaN,GK,Artur Boruc
2,2,NaN,DF,Simon Francis (captain)[31]
3,3,NaN,DF,Steve Cook (2nd vice-captain)
4,4,NaN,MF,Dan Gosling
5,5,NaN,DF,Nathan Aké
6,6,NaN,MF,Andrew Surman (vice-captain)[31]
7,7,NaN,MF,Marc Pugh
8,8,NaN,MF,Jefferson Lerma
9,9,NaN,FW,Lys Mousset


In [61]:
dfs[2]

,0,1,2,3
0,No.,NaN,Position,Player
1,13,NaN,FW,Callum Wilson
2,15,NaN,DF,Adam Smith
3,16,NaN,MF,Lewis Cook
4,17,NaN,FW,Joshua King
5,18,NaN,FW,Jermain Defoe
6,19,NaN,MF,Junior Stanislas
7,20,NaN,MF,David Brooks
8,21,NaN,DF,Diego Rico
9,24,NaN,MF,Ryan Fraser


#### Club official info

In [79]:
# Must check the number of columns
club1_officials=parse_html_table(alltables_club1[7], alltables_club1[7])
club1_officials

,Position,Name
0,Chairman,Jeff Mostyn
1,Chief Executive,Neill Blake
2,Manager,Eddie Howe
3,Assistant Manager,Jason Tindall
4,First Team Coach,Stephen Purches
5,First Team Coach,Simon Weatherstone
6,Assistant First Team Coach,Steve Fletcher
7,Development Squad/U21 Manager,Carl Fletcher
8,Head Goalkeeper Coach,Neil Moss
9,First Team Assistant Goalkeeper Coach,Anthony White


## Player data

Retrieve data on each individual player.

What kind of data and what format?

### Check for disambiguation page

In [4]:
TITLE_PLAYER = 'Simon Francis'

PARAMS_DISAMB = {
    'action': 'query', # the action is to parse the content of the page
    'titles': TITLE_PLAYER,
    'format': 'json', # the API can return other formats but they are standardising to JSON
    'prop': 'categories'
}

# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS_DISAMB)
player_1_query = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

In [5]:
#  Convert to string in order to easily test if contains specific string
player_1_query = json.dumps(player_1_query)

if 'Category:All disambiguation pages' in player_1_query:
    TITLE_PLAYER = TITLE_PLAYER + ' (footballer)'

In [6]:
TITLE_PLAYER

'Simon Francis (footballer)'

### Retrieve player info


In [211]:
'''
PARAMS_PLAYER = {
    'action': 'parse', # the action is to parse the content of the page
    'page': TITLE_PLAYER,
    'format': 'json' # the API can return other formats but they are standardising to JSON
}


# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS_PLAYER)
DATA_PLAYER = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

# Write to JSON file
with open('player_data.json', 'w') as f:
    json.dump(DATA_PLAYER, f)
'''

In [7]:
# Read JSON file
with open('player_data.json') as data_file:
    DATA_PLAYER = json.load(data_file)

In [8]:
type(DATA_PLAYER)

dict

In [9]:
# Place the first team page data into bs4
soup_player1 = BeautifulSoup(DATA_PLAYER['parse']['text']['*'], "html.parser")

alltables_player1 = soup_player1.findAll("table")

# Redundent code
#html_content = str(alltables_player1[1])
#fs_player = pd.read_html(html_content, header = 0, tupleize_cols=True, flavor = 'bs4')

In [19]:
# Source of code: https://stackoverflow.com/questions/28763891/what-should-i-do-when-tr-has-rowspan
def pre_process_table(table):
    """
    INPUT:
        1. table - a bs4 element that contains the desired table: ie <table> ... </table>
    OUTPUT:
        a tuple of: 
            1. rows - a list of table rows ie: list of <tr>...</tr> elements
            2. num_rows - number of rows in the table
            3. num_cols - number of columns in the table
    Options:
        include_td_head_count - whether to use only th or th and td to count number of columns (default: False)
    """
    rows = [x for x in table.find_all('tr')]

    num_rows = len(rows)

    # get an initial column count. Most often, this will be accurate
    num_cols = max([len(x.find_all(['th','td'])) for x in rows])

    # sometimes, the tables also contain multi-colspan headers. This accounts for that:
    header_rows_set = [x.find_all(['th', 'td']) for x in rows if len(x.find_all(['th', 'td']))>num_cols/2]

    num_cols_set = []

    for header_rows in header_rows_set:
        num_cols = 0
        for cell in header_rows:
            row_span, col_span = get_spans(cell)
            num_cols+=len([cell.getText()]*col_span)

        num_cols_set.append(num_cols)

    num_cols = max(num_cols_set)

    return (rows, num_rows, num_cols)


def get_spans(cell):
        """
        INPUT:
            1. cell - a <td>...</td> or <th>...</th> element that contains a table cell entry
        OUTPUT:
            1. a tuple with the cell's row and col spans
        """
        if cell.has_attr('rowspan'):
            rep_row = int(cell.attrs['rowspan'])
        else: # ~cell.has_attr('rowspan'):
            rep_row = 1
        if cell.has_attr('colspan'):
            rep_col = int(cell.attrs['colspan'])
        else: # ~cell.has_attr('colspan'):
            rep_col = 1 

        return (rep_row, rep_col)

def process_rows(rows, num_rows, num_cols):
    """
    INPUT:
        1. rows - a list of table rows ie <tr>...</tr> elements
    OUTPUT:
        1. data - a Pandas dataframe with the html data in it
    """
    data = pd.DataFrame(np.ones((num_rows, num_cols))*np.nan)
    for i, row in enumerate(rows):
        try:
            col_stat = data.iloc[i,:][data.iloc[i,:].isnull()].index[0]
        except IndexError:
            print(i, row)

        for j, cell in enumerate(row.find_all(['td', 'th'])):
            rep_row, rep_col = get_spans(cell)

            #print("cols {0} to {1} with rep_col={2}".format(col_stat, col_stat+rep_col, rep_col))
            #print("\trows {0} to {1} with rep_row={2}".format(i, i+rep_row, rep_row))

            #find first non-na col and fill that one
            while any(data.iloc[i,col_stat:col_stat+rep_col].notnull()):
                col_stat+=1

            data.iloc[i:i+rep_row,col_stat:col_stat+rep_col] = cell.getText().strip()
            if col_stat<data.shape[1]-1:
                col_stat+=rep_col

    return data

def main(table):
    rows, num_rows, num_cols = pre_process_table(table)
    df = process_rows(rows, num_rows, num_cols)
    return(df)

In [82]:
table = alltables_player1[1]
## run the above functions to extract the data
rows, num_rows, num_cols = pre_process_table(table)
df = process_rows(rows, num_rows, num_cols)

# Re-do the formatting
df = df[~df[1].isin(['Total', 'Career total', 'Season'])] # Remove rows containing totals and headers
df = df.drop([11, 12], axis = 1) # Remove columns containing totals

# Re-do the headers
player_cols = ['Club', 'Season','Division', 'League - Apps', 'League - Goals', 'FA Cup - Apps', 'FA Cup - Goals', 'League Cup - Apps', 'League Cup - Goals', 'Other - Apps', 'Other - Goals']
df.columns = player_cols # Apply the correct headers

# Remove link references from 'Season' column
df['Season'] = df['Season'].str.extract(r"^(\d{4}–\d{2})", expand = False)

# TO DO: convert data type of Season column to date/year

df = df.reset_index(drop=True)
df.head()

,Club,Season,Division,League - Apps,League - Goals,FA Cup - Apps,FA Cup - Goals,League Cup - Apps,League Cup - Goals,Other - Apps,Other - Goals
0,Bradford City,2002–03,First Division,25,1,0,0,1,0,0,0
1,Bradford City,2003–04,First Division,30,0,0,0,1,0,0,0
2,Sheffield United,2003–04,First Division,5,0,0,0,0,0,0,0
3,Sheffield United,2004–05,Championship,6,0,1,0,0,0,0,0
4,Sheffield United,2005–06,Championship,1,0,0,0,2,0,0,0
